In [97]:
import numpy as np
from sklearn import preprocessing
import os.path
import librosa
import scipy.stats as stats
from sklearn.preprocessing import MinMaxScaler

#### **2. Extracção de Features.**
2.1. Processar as features do ficheiro top100_features.csv.

2.1.1. Ler o ficheiro e criar um array numpy com as features disponibilizadas.

In [98]:
def extract_features():
    data = np.genfromtxt("Features/top100_features.csv", delimiter = ",", skip_header = 1)
    data = data[:, 1:-1]
    return data


2.1.2. Normalizar as features no intervalo [0, 1].

In [99]:
def normalize_features(data):
    max_values = data.max(axis = 0)
    min_values = data.min(axis = 0)

    return (data-min_values)/(max_values - min_values)

2.1.3. Criar e gravar em ficheiro um array numpy com as features extraídas (linhas = músicas; colunas = valores das features).

In [100]:
def save_top100_features_normalized(path, data_normalized):
    np.savetxt(path, data_normalized, delimiter=",", fmt="%.6f")

2.2. Extrair features da framework librosa. <br>

2.2.1. Para os 900 ficheiros da BD, extrair as seguintes features (sugestão: guardar
todas as músicas na mesma pasta):
- Features Espectrais: mfcc, spectral centroid, spectral bandwidth, spectral
contrast, spectral flatness e spectral rolloff.
- Features Temporais: F0, rms e zero crossing rate.
- Outras features: tempo.
- Utilize os parâmetros por omissão do librosa (sr = 22050 Hz, mono, window
length = frame length = 92.88 ms e hop length = 23.22 ms).
- Guarde as features num array numpy 2D, com número de linhas = número de
músicas e número de colunas = número de features

In [101]:
def extract_music_features(music):
    path = "Music/"
    
    data, sampling_rate = librosa.load(path + music)

    mfcc = librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc = 13)
    spectral_centroid = librosa.feature.spectral_centroid(y=data, sr=sampling_rate)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=data, sr=sampling_rate)
    spectral_contrast = librosa.feature.spectral_contrast(y=data, sr=sampling_rate)
    spectral_flatness = librosa.feature.spectral_flatness(y=data)
    spectral_rolloff = librosa.feature.spectral_rolloff(y = data, sr = sampling_rate)
    f0 = librosa.yin(y=data, fmin=20, fmax=11025)
    f0[f0==11025] = 0  
    rms = librosa.feature.rms(y=data)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=data)
    tempo = librosa.beat.tempo(y=data, sr=sampling_rate)
        
    return mfcc, spectral_centroid, spectral_bandwidth, spectral_contrast, spectral_flatness, spectral_rolloff, f0, rms, zero_crossing_rate, tempo

2.2.2. Calcular as 7 estatísticas típicas sobre as features anteriores: média, desvio
padrão, assimetria (skewness), curtose (kurtosis), mediana, máximo e mínimo.
Para o efeito, utilizar a biblioteca scipy.stats (e.g., scipy.stats.skew).

In [102]:
def calculate_statistics(feature, axis_):
    feat_mean = feature.mean(axis = axis_)
    feat_std = feature.std(axis = axis_)
    feat_skewness = stats.skew(feature, axis = axis_)
    feat_kurtosis = stats.kurtosis(feature, axis = axis_)
    feat_median = np.median(feature, axis = axis_)
    feat_min = feature.min(axis = axis_)
    feat_max = feature.max(axis = axis_)
    
    if(axis_ == 1):
        return np.hstack((feat_mean, feat_std, feat_skewness, feat_kurtosis, feat_median, feat_min, feat_max))
    else:
        # f0
        return np.hstack(([feat_mean], [feat_std], [feat_skewness], [feat_kurtosis], [feat_median], [feat_min], [feat_max]))
    

2.2.3. Normalizar as features no intervalo [0, 1].

In [103]:
# Chamamos a função normalize_features()

2.2.4. Criar e gravar em ficheiro o array numpy com as features extraídas.

#### **Main function**

In [104]:
if __name__ == "__main__" :
    saved_features = "top100_features_normalized.csv"
    librosa_features = "librosa_features.csv"
    music_folder = os.listdir("Music")    

    # Ex 2.1
    
    if os.path.exists(saved_features):
        data_normalized = np.genfromtxt(saved_features, delimiter = ",")
    else:
        data_extracted = extract_features()
        data_normalized = normalize_features(data_extracted)
        save_top100_features_normalized(saved_features, data_normalized)
    
    
    # Ex 2.2
    if os.path.exists(librosa_features):
        librosa_data_normalized = np.genfromtxt(librosa_features, delimiter = ",")
    else:
        statistics = np.empty((0, 190))
        total_normalized_statistics = np.array([])
        
        iteration = 1
        for music in music_folder:
            print(iteration)
            mfcc, spectral_centroid, spectral_bandwidth, spectral_contrast, spectral_flatness, spectral_rolloff, f0, rms, zero_crossing_rate, tempo = extract_music_features(music)
            
            mfcc_statistics = calculate_statistics(mfcc, 1)
            spectral_centroid_statistics = calculate_statistics(spectral_centroid, 1)
            spectral_bandwidth_statistics = calculate_statistics(spectral_bandwidth, 1)
            spectral_contrast_statistics = calculate_statistics(spectral_contrast, 1)
            spectral_flatness_statistics = calculate_statistics(spectral_flatness, 1)
            spectral_rolloff_statistics = calculate_statistics(spectral_rolloff, 1)
            f0_statistics = calculate_statistics(f0, 0)
            rms_statistics = calculate_statistics(rms, 1)
            zero_crossing_rate_statistics = calculate_statistics(zero_crossing_rate, 1)
            
            music_statistics = np.concatenate((mfcc_statistics.flatten(), 
                                                spectral_centroid_statistics,
                                                spectral_bandwidth_statistics,
                                                spectral_contrast_statistics.flatten(),
                                                spectral_flatness_statistics,
                                                spectral_rolloff_statistics,
                                                f0_statistics, 
                                                rms_statistics,
                                                zero_crossing_rate_statistics,
                                                tempo))
            statistics = np.vstack((statistics, music_statistics))
            iteration += 1
            
        librosa_data_normalized = normalize_features(statistics)
        save_top100_features_normalized(librosa_features, librosa_data_normalized)

1


C:\Python39\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


2
3
4
5
6
7
8
